## Extraktionsbaserad textsammanfattare med olika rankningsmått 

#### Imports

In [1]:
# Imports --> TODO fixa ordning 
# Web scrapping -->  module för att ladda ner artiklar 
from newspaper import Article
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk 
import ssl
import re  # används ej någonstans (tror jag) / björn
import regex as rex # 


# Summarization length of original text
percentage = 0.15
n_sentences = 3

# Fixes some errors, found online at https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#### Input

## Lägg till nya artiklar 

* Hämta komplett url 
* Lägg till i python cellen under här
* Tilldela till variabeln text 
* Tryck Restart och sen Run All 
* Filerna hamnar i mappen "summarizations"

In [2]:
#text = "https://www.aftonbladet.se/nyheter/a/kE6ExL/sd-far-tunga-poster-i-utskotten"
#text = 'https://www.svt.se/nyheter/utrikes/stall-dina-fragor-om-kriget-till-svt-s-utrikesreportrar'
#text = "https://www.aftonbladet.se/nyheter/a/kE6j0a/uppgifter-viktigt-fynd-i-jakten-pa-mordarna-i-sodertalje"
text = 'https://www.aftonbladet.se/nyheter/a/EQaJRo/blixtbygget-ska-radda-tyskland-i-vinter'

In [3]:
article = Article(text, language='sv')
article.download()
article.parse()
text = article.text
homepage = article.meta_data['al']['ios']['app_name']
link = article.url
title = article.title

# Beroende på vilken hemsida nyheten kommer ifrån kan titeln och texten inehålla delar av sidan man egentligen inte bryr sig om
# T.ex. från aftonbladet är titeln med i texten och texten innehåller en mening som: "publicerad: 30 sep", man kan ta bort detta men 
# det blir om vi får tid över.

#print('Title:' , article.title, '\n\nText: \n', text)

# Preprocessing 

## Overview
### Calculate number of sentences to keep

### List 1 - sentences
* Varje mening separat

### Dataframe - scores
#### columns are the score of each ranking measure
* Baseline
* Headings
* TF/IDF-score
* NER
* ~~Class~~ //Om vi har tid för ML

### List 2 -> Cleaned for Stop Words 
* Varje mening separat 

###

##### Original Sentences List

In [4]:
# removes endlines:
from token import NEWLINE

org_sentences = text.replace('\n\n', '. ')
# creates some exceptions from above rule
org_sentences = org_sentences.replace('.. ', '. ')
org_sentences = org_sentences.replace(':. ', ': ')
org_sentences = org_sentences.split('. ')

sentences_to_remove = []
for i, sentence in enumerate(org_sentences): 
    if sentence == title: 
        sentences_to_remove.append(sentence)
    if 'publicerad:' in sentence.lower(): 
        sentences_to_remove.append(sentence)
    if 'uppdaterad:' in sentence.lower(): 
        sentences_to_remove.append(sentence)

org_sentences = [sentence for sentence in org_sentences if sentence not in sentences_to_remove]

org_sentences[0:5]

['Bygget längst ut på en flera hundra meter lång pir ser inte mycket ut för världen',
 'Ett antal breda, rostfärgade rör som förankrats i havsbotten',
 'Plattformar ovanpå',
 'Men den här terminalen för flytande naturgas ska rädda tyskarna från att slippa frysa – när vintern kommer',
 'WILHELMSHAVEN, TYSKLAND Den byggs i rekordfart']

##### Dataframe

In [5]:
index = range(org_sentences.__len__())
columns = ['Baseline', 'Headings', 'TF', 'NER']
scores = pd.DataFrame(index=index, columns=columns)
scores.fillna(0, inplace=True)
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Baseline  108 non-null    int64
 1   Headings  108 non-null    int64
 2   TF        108 non-null    int64
 3   NER       108 non-null    int64
dtypes: int64(4)
memory usage: 3.5 KB


#### Create spacy doc object

In [6]:
import spacy
# Credit to Explosion for sv_core_news_sm --> https://github.com/explosion 
# "lemmatization accuracy 0.95"
# Create spacy nlp object 
nlp = spacy.load("sv_core_news_sm") # nlp used by lemmatizer()

#### Lemmatizer

In [7]:
# den --> det 
# noterar detta att det är en dålig lemmatiserare --> språkbanken stanza / lemmy / kth...  

def lemmatizer(list_of_strings):
    lemmatized_sentences = []
    lemmatized_sentence = ''
    for i in range(len(list_of_strings)): 
        sentence_to_lemmatize = nlp(list_of_strings[i])
        for token in sentence_to_lemmatize:
            lemma = token.lemma_
            lemmatized_sentence += lemma + ' '  
        
        lemmatized_sentences.append(lemmatized_sentence)
        lemmatized_sentence = '' 

    return lemmatized_sentences

# Created Lemmatized DS
lemmatized_org_sentences = lemmatizer(org_sentences)
print(lemmatized_org_sentences)

['Bygge länge ut på en flera hundra meter lång pir se inte mycket ut för värld ', 'en antal bred , rostfärga röra som förankra i havsbott ', 'plattformar ovanpå ', 'men en här terminal för flyta naturgas skola rädda tysk från att slippa frysa – när vinter komma ', 'WILHELMSHAVEN , TYSKLAND den bygga i rekordfar ', 'normal ta en sån här anläggning 5–6 år att få klar enligt energibolagen Unipers operative chef Holger kreetz ', 'men genom att snabba på tillståndsprocess och kapa en antal byråkratisk led vara förhoppning att den skola ta lite än tio månad ', 'vi befinna vi i norra Tyskland , vid Yttre hamn i Wilhelmshaven ', 'på ena sida Nordsjön , på en annan en platt landskap med omväxlande industri och lång sandstränd ', 'i slut av december skola en en fartyg som leverera flyta naturgas , så kalla LNG , kunna lägga till vid terminal ', 'en skäl till att den gå så fort vara att en tysk regering ge Uniper i uppdrag att bygga terminal och dessutom drivit igenom en särskild lagstiftning som

#### Proper Nouns

In [8]:
# För bättre täckning på NER 
def proper_nouns(list_of_strings):
    proper_nouns = set()
    for i in range(len(list_of_strings)): 
        sentence_to_pos = nlp(list_of_strings[i])
        for token in sentence_to_pos: 
            token_str = token.text
            if token.pos_ == "PROPN" and len(token_str) > 1:
                proper_nouns.add(token_str.strip())
    return proper_nouns

print(proper_nouns(org_sentences))


{'LNG', 'Wilhelmshaven', 'Nord', 'USA', 'Keertz', 'Hansson', 'Wolfgang', 'Oliver', 'Kina', 'Holger', 'Rädsla', 'Australien', 'Putin', 'Ryssland', 'Berlin', 'Nigeria', 'Ukraina', 'Ivarsson', 'Jerker', 'WILHELMSHAVEN', 'Zaporizjzja', 'TYSKLAND', 'Bund', 'Wuhan', 'Unipers', 'Tyskland', 'Europa', 'Nordsjön'}


#### Named Entities

In [9]:
def named_entity_recognition(list_of_strings):
    doc = nlp(' '.join(list_of_strings))
    # Convert tuple[Span] to str
    named_entities = doc.ents.__str__()
    # Remove string parenthesis 
    named_entities = named_entities[1:len(named_entities) - 1]
    # Create list of strings
    named_entities = named_entities.split(',')
    
    named_entities_set = set()
    for entity in named_entities: 
        named_entities_set.add(entity.strip())
    return named_entities_set

### Stop Word Filtering

In [10]:
# Inspired by https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

def get_swe_stop_words(): 
    swe_stop_words = set(stopwords.words('swedish'))
    # Not stop word cleaning 
    swe_stop_words.update([',', '"', ':', '-', '–', '”'])
    return swe_stop_words

def stop_word_filtering(list_of_strings):
    word_tokens = word_tokenize(' '.join(list_of_strings))
    filtered_sentence_into_singletons = [w for w in word_tokens if not w.lower() in get_swe_stop_words()]
    return filtered_sentence_into_singletons

#### Frequency Distribution 

In [11]:
def frequency_distribution(list): 
    fdist = FreqDist(word.lower() for word in word_tokenize(' '.join(list)))
    return fdist

In [12]:
filtered_words = stop_word_filtering(lemmatized_org_sentences) 
fdist = frequency_distribution(filtered_words)

# Ranking Measures

### Baseline
(1/N --> n=ordning mening kommer i dvs. första meningen får N=1 -> 1/1, andra meningen får N=2 -> 1/2, osv.)

In [13]:
# Ranking metric 1 --> Baseline
for i, score in enumerate(scores['Baseline']) :
    scores['Baseline'][i] = 1/((i+1))
scores.describe()

,Baseline,Headings,TF,NER
count,108.000000,108.0,108.0,108.0
mean,0.048740,0.0,0.0,0.0
std,0.113531,0.0,0.0,0.0
min,0.009259,0.0,0.0,0.0
25%,0.012308,0.0,0.0,0.0
50%,0.018350,0.0,0.0,0.0
75%,0.036045,0.0,0.0,0.0
max,1.000000,0.0,0.0,0.0


### Headings

In [14]:
# Ranking metric 2 --> Headings
# Sets all 'Headings' scores to 0, mostly for testing so i can run this multiple times, 
# but also to make sure nothing weird has happened earlier in the code.
scores['Headings'] = 0
for i, sentence in enumerate(org_sentences): # Lemmatizerad? 
    for word in article.title.split(' '):
        if word in sentence:
            scores.at[i, 'Headings'] += 1
scores.describe()

,Baseline,Headings,TF,NER
count,108.000000,108.00000,108.0,108.0
mean,0.048740,0.87963,0.0,0.0
std,0.113531,0.92441,0.0,0.0
min,0.009259,0.00000,0.0,0.0
25%,0.012308,0.00000,0.0,0.0
50%,0.018350,1.00000,0.0,0.0
75%,0.036045,1.00000,0.0,0.0
max,1.000000,4.00000,0.0,0.0


Sverigedemokraterna få ordförandepost i riksdag justitie och utrikesutskott 
[' få ', 'ordförandepost ', 'riksdag ', 'utrikesutskott ']

* bara ord i lemma 
* ord i lemma med efter letters
* ord i lemma med före letters --> x
* före + lemma + efter 

In [15]:
preprocessed_title = stop_word_filtering(lemmatizer(title.split(' ')))
p = rex.compile(r"\b\S*\L<words>\S*\b | \b\L<words>\b | \b\L<words>\S*\b", re.IGNORECASE, words=preprocessed_title)
p.findall

for i, sentence in enumerate(lemmatized_org_sentences):    
    #print(sentence)
    matches = p.findall(sentence)
    #print(matches)
    scores.at[i, "Headings"] = len(matches)

scores.describe()

,Baseline,Headings,TF,NER
count,108.000000,108.000000,108.0,108.0
mean,0.048740,0.981481,0.0,0.0
std,0.113531,1.102092,0.0,0.0
min,0.009259,0.000000,0.0,0.0
25%,0.012308,0.000000,0.0,0.0
50%,0.018350,1.000000,0.0,0.0
75%,0.036045,1.000000,0.0,0.0
max,1.000000,5.000000,0.0,0.0


## TERM FREQUENCY 


In [16]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
#Term Frequency
def termfreq(sentence, word):
    N = len(sentence)
    occurance = len([token for token in sentence if token == word])
    return occurance/N

In [17]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
#Inverse Document Frequency
def inverse_doc_freq(word):
    try:
        word_occurance = fdist.get(word) + 1
    except:
        word_occurance = 1
    return np.log(len(lemmatized_org_sentences)/word_occurance)

In [18]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
# TF*IDF-score 
def tf_idf(sentence):
    score = 0
    #tf_idf_vec = np.zeros((len(filtered_words),))
    for word in sentence:
        tf = termfreq(sentence,word)
        idf = inverse_doc_freq(word)
         
        score += tf*idf
        #tf_idf_vec[index_dict[word]] = value 
    return score

for i, sentence in enumerate(lemmatized_org_sentences):
    scores.at[i, 'TF'] = tf_idf(sentence)

#scores.describe()
#scores['TF'].idxmax()
#scores.loc[scores['TF'] == scores['TF'].median()]

### TF*IDF-score

https://forketyfork.medium.com/latex-math-formulas-a-cheat-sheet-21e5eca70aae

In [19]:
# Ranking metric 3 --> TF*IDF

# Term Weights --> Calculate importance of single words in text/doc
# Binary term weights --> document specific
# TF*IDF term weights --> document-collection specific 

# Assign weights to each dimension (attr/word) of each sentence (record/example) 

# Term Frequency (TF-score) --> TFij == frequency of the jth term in in the ith doc 

# Inverse Document Frequency 
# idf-score of the jth term measures the uniqueness of the jth term in the collection of documents
# IDFj = log(M / Nj)
#
# M = total num of docs in collection 
# Nj is the number of documents that contain the jth term

# HIGH TF*IDF-score 
# Word frequent in document && Occur in few documents of the collection 
# LOW TF*IDF-score
# Not present in document || present in all documents of the collection 

### NER 

In [20]:
scores['NER'] = 0
named_entities = named_entity_recognition(org_sentences)
proper_nouns = proper_nouns(org_sentences)
ner_unique = named_entities.union(proper_nouns)
print(ner_unique)

{'LNG', 'USA', 'Keertz', 'Wilhelmshaven', 'Nord', 'Wolfgang Hansson', 'nyligen', 'Hansson', 'Wolfgang', 'Oliver', 'Nord Stream 1', 'Kina', 'Holger', 'Nu', 'för ett år sedan', 'i framtiden', 'Rädsla', 'Australien', 'Putin', 'Habeck', 'Ryssland', 'Robert', 'Nigeria', 'Kreetz', 'Ukraina', 'Berlin', 'Ivarsson', 'Jerker', 'WILHELMSHAVEN', 'Zaporizjzja', 'TYSKLAND', 'Uniper', 'Bund', 'Wuhan', 'Unipers', 'Olaf', 'Tyskland', 'Yttre', 'Plattformar', 'Oliver Powalla', 'På ett år', 'Europa', 'Jerker Ivarsson', 'De gröna', 'Holger Kreetz', 'Nordsjön'}


In [21]:
p = rex.compile(r"\L<words>", words=ner_unique)

for i, sentence in enumerate(org_sentences):
    matches = p.findall(sentence)
    scores.at[i, "NER"] = len(matches)

scores.describe()
scores.head()


,Baseline,Headings,TF,NER
0,1.000000,0,30.829098,0
1,0.500000,1,22.729619,0
2,0.333333,0,8.624979,1
3,0.250000,2,36.057237,0
4,0.200000,1,11.854758,2


# Combination Function

#### Standardize

In [22]:
# Standardize
scores_standardized = StandardScaler.fit_transform(self=StandardScaler(), X=scores)
scores_standardized = pd.DataFrame(scores_standardized, columns=columns)
scores_standardized['Baseline'] *= 1.5
scores_standardized['Headings'] *= 2.5
scores_standardized['TF'] *= 0.3
scores_standardized

# ÄNDRA COL NAME TF col --> TF*IDF TODO 
# COUNTER WEIGHT: 
# IF -->  
# Baseline score low (e.g. below median)
# && term_frequency high (note: not TF*IDF) (e.g. above upper quartile)
# && NER high (e.g. above upper quartile)
# THEN: 
# add_counterweight(sentence, counter_weight) 
# counter_weight:  
# final rankposition = i -->  ∑ -1 (-1 may be too big weight)

,Baseline,Headings,TF,NER
0,12.626894,-2.236785,0.010758,-0.722652
1,5.989960,0.042203,-0.162605,-0.722652
2,3.777648,-2.236785,-0.464503,0.195541
3,2.671493,2.321192,0.122663,-0.722652
4,2.007799,0.042203,-0.395372,1.113735
...,...,...,...,...
103,-0.519341,-2.236785,-0.188114,-0.722652
104,-0.520557,-2.236785,-0.276878,-0.722652
105,-0.521749,0.042203,-0.308375,1.113735
106,-0.522920,-2.236785,-0.375187,-0.722652


#### Calculate Summarization Length (number of sentences)

In [23]:
num_of_org_sentences = len(org_sentences)
summarization_num_sentences = round(num_of_org_sentences * percentage)

print("summarization: ", summarization_num_sentences, "\noriginal: ", num_of_org_sentences)

summarization:  16 
original:  108


#### Combine
* Combine the scores into one overall score
* add weight and/or ML if time allows

In [24]:
# Combination Function
# Här ligger ML om vi gör det  

#final_score = scores_standardized.drop('TF', axis=1).sum(axis=1)
final_score = scores_standardized.sum(axis=1)
best_sentences = final_score.nlargest(summarization_num_sentences, keep='all').index.values
print(best_sentences)

print(org_sentences[0])


[ 57  12   0 100  15  18  35   1  65  16   3  59   9  51  34   6]
Bygget längst ut på en flera hundra meter lång pir ser inte mycket ut för världen


In [25]:
# Similarity heurstic 
# Rubrik --> Hur mycket lika --> Ta bort redundans 
# 5 fem väldigt lika --> heuristik välja bort något av det     

# Assemble output 
* Reassemble according to overall score ranking
* Output summarization 

In [26]:
# Percentage length of original text --> summarization 
print("Percentage:\n")
print(title, "\n")
for i in best_sentences: 
    print(org_sentences[i])

print("\n\n")

# N sentences 
print("N sentences:\n")
for i in best_sentences[0:n_sentences]: 
    print(org_sentences[i])

Percentage:

Putin stängde av gasen – då tog Tyskland ett blixtbeslut 

Men ekonomiminister Robert Habeck från De gröna har sagt att klimataspekten i det här fallet får stå åt sidan för den viktigare uppgiften att förhindra president Putin att fortsätta utpressa Tyskland med sitt gasvapen
Nu har president Putin stängt av gasen nästan helt och sprängningarna av Nord Stream 1 och 2 har gjort att det inte ens i teorin går att återuppta gasleveranserna i närtid
Bygget längst ut på en flera hundra meter lång pir ser inte mycket ut för världen
Det är gasledningen som ska ta LNG-gasen fram till där det går att ansluta till det tyska gasnätet
Den nedkylda och kraftigt komprimerade flytande gasen omvandlas sedan till vanlig gas som via pipelines förs ner i det redan existerande tyska gasnätet
– Bara vid den här terminalen kommer vi att kunna ersätta 20 procent av den gas som Tyskland tidigare fått från Ryssland, säger Unipers operative chef Holger Kreetz
Invasionen av Ukraina har gett en helt n

In [27]:
# Newspaper Summarization
article.nlp()
print("\nNewspaper3k: \n", article.summary)


Newspaper3k: 
 Men genom att snabba på tillståndsprocessen och kapa ett antal byråkratiska led är förhoppningen att det ska ta mindre än tio månader.
I stället ser han energieffektivisering och sparade som de enskilt viktigaste åtgärderna för att Tyskland ska klara sig igenom vintern.
Precis innan vi lämnar kajen för den korta båtresan ut till piren ligger ett stort krigsfartyg ute på redden.
Hur ska det exempelvis gå att fylla lagren inför nästa vinter?
Det är gasledningen som ska ta LNG-gasen fram till där det går att ansluta till det tyska gasnätet.


In [28]:
summarization = link + '\n\n' + 'Title: ' + '\n' + title + '\n\n' + 'Summarization:' + '\n'
for sentence in best_sentences: 
    summarization += org_sentences[sentence] + '\n'
summarization += '\n\n' + 'Newspaper3k:' + '\n' + 'Title: ' + '\n' + title + "\n\n" + article.summary

In [29]:
# import os 
# FILEPATH = f'./summarizations/{homepage}/'
# filename = f"{FILEPATH}{title}.txt"
# os.makedirs(os.path.dirname(filename), exist_ok=True)

# with open(filename, 'w') as f:
#      f.write(summarization)